In [1]:
import pandas as pd
import numpy as np

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import json

from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# LOADING CSV FILES

In [2]:
df = pd.read_csv("data/final/final_dataset.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7061 entries, 0 to 7060
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   6295 non-null   float64
 1   aliases                      0 non-null      float64
 2   api_detail_url               7061 non-null   object 
 3   associated_images            7061 non-null   object 
 4   character_credits            7061 non-null   object 
 5   character_died_in            7061 non-null   object 
 6   concept_credits              7061 non-null   object 
 7   cover_date                   7061 non-null   object 
 8   date_added                   7061 non-null   object 
 9   date_last_updated            7061 non-null   object 
 10  deck                         475 non-null    object 
 11  description                  6982 non-null   object 
 12  first_appearance_characters  0 non-null      float64
 13  first_appearance_c

# PREPROCESSING & FEATURE ENGINEERING FUNCTIONS

**clean_dataframe** <br>
input: df <br>
drops various columns with too many nulls, converts some types to datetime and fill in null descriptions with "" <br>
also drops various columns I deem aren't needed for the final model <br>
output: df <br>

In [4]:
def clean_dataframe(input_dataframe):
    df = input_dataframe.copy()
    df.drop(columns=["Unnamed: 0", "aliases", "deck", "first_appearance_characters", "first_appearance_concepts", 
                    "first_appearance_locations", "first_appearance_objects", "first_appearance_storyarcs",
                    "first_appearance_teams", "store_date"], inplace=True)
    df.dropna(subset=['name'], inplace=True)
    df['cover_date'] = pd.to_datetime(df['cover_date'], errors='coerce')
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
    df['date_last_updated'] = pd.to_datetime(df['date_last_updated'], errors='coerce')
    df.drop(columns=['api_detail_url', 'associated_images', 'date_added', 'date_last_updated', 'image',
                    'site_detail_url'], inplace=True)
    df['description'] = df['description'].fillna('')
    #df = df.reset_index()
    return df

**clean_description**
cleans the description as well as possible
the format and contents of the descriptions are not always consistent so there are some errors

In [5]:
def clean_description(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    cleaned_text = soup.get_text(separator='\n')
    cleaned_text = cleaned_text.replace('\\', '').replace('\n', '')
    return cleaned_text

**get_names and get_obj** <br>
get_names takes in lists and get_obj is for when there is only one object <br>
when there is an error, i've set it to return an empty list ([])

In [6]:
def get_names(json_str, index):
    json_str = json_str.replace("'",'"')
    try:
        lst = json.loads(json_str)
    except json.JSONDecodeError as e:
        #print(f"JSONDecodeError at row {index}: {e}")
        return []
    names = [item["name"] for item in lst]
    return names
def get_obj(json_str, index):
    json_str = json_str.replace("'",'"')
    try:
        obj = json.loads(json_str)
    except json.JSONDecodeError as e:
        #print(f"JSONDecodeError at row {index}: {e}")
        return []
    names = [obj["name"]]  # Since we are dealing with a single object, not a list
    return names

**total_clean** applies all the above functions

In [7]:
def total_clean(df):
    df["description"] = df["description"].apply(clean_description)
    df['character_credits'] = df.apply(lambda row: get_names(row['character_credits'], row.name), axis=1)
    df['character_died_in'] = df.apply(lambda row: get_names(row['character_died_in'], row.name), axis=1)
    df['concept_credits'] = df.apply(lambda row: get_names(row['concept_credits'], row.name), axis=1)
    df['location_credits'] = df.apply(lambda row: get_names(row['location_credits'], row.name), axis=1)
    df['object_credits'] = df.apply(lambda row: get_names(row['object_credits'], row.name), axis=1)
    df['person_credits'] = df.apply(lambda row: get_names(row['person_credits'], row.name), axis=1)
    df['story_arc_credits'] = df.apply(lambda row: get_names(row['story_arc_credits'], row.name), axis=1)
    df["team_credits"] = df.apply(lambda row: get_names(row['team_credits'], row.name), axis=1)
    df['team_disbanded_in'] = df.apply(lambda row: get_names(row['team_disbanded_in'], row.name), axis=1)
    df['volume'] = df.apply(lambda row: get_obj(row['volume'], row.name), axis=1)

**to_underscore** replaces the spaces with underscore in the credits <br>
this is in preparation for when we vectorize and to make sure that we don't seperate the first and last names

In [8]:
def to_underscore(df):
    df['character_credits'] = df['character_credits'].apply(lambda x:[name.replace(' ', '_') for name in x])
    df['character_died_in'] = df['character_died_in'].apply(lambda x:[name.replace(' ', '_') for name in x])
    df['concept_credits'] = df['concept_credits'].apply(lambda x:[name.replace(' ', '_') for name in x])
    df['location_credits'] = df['location_credits'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['object_credits'] = df['object_credits'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['person_credits'] = df['person_credits'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['story_arc_credits'] = df['story_arc_credits'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['team_credits'] = df['team_credits'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['team_disbanded_in'] = df['team_disbanded_in'].apply(lambda x: [name.replace(' ', '_') for name in x])
    df['volume'] = df['volume'].apply(lambda x: [name.replace(' ', '_') for name in x])

**to_string** converts the various credits into from lst to string <br>
**combined_description** combines the description col with all the credits cols to create a new feature 

In [9]:
def to_string(lst):
    string = ""
    for item in lst:
        string = string + " " + item
    return string

In [10]:
def combined_description(df):
    lst = ['character_credits', 'concept_credits', 'location_credits', 'object_credits', 
           'person_credits', 'story_arc_credits', 'team_credits', 'volume']
    df['combined_description'] = df['description']
    for col in lst:
        df['combined_description'] = df['combined_description'] + df[col].apply(to_string)

# ACTUALLY DOING THE PREPROCESSING & FEATURE ENGINEERING

In [11]:
#dropping stuff and converting datetime and filling in nulls
df = clean_dataframe(df)

In [12]:
#cleaning text and getting lists for various credits
total_clean(df)

In [13]:
#replcing spaces with underscores in various credits
to_underscore(df)

In [14]:
#combining desciprtion and various credits
combined_description(df)

In [15]:
df.sample(3)

,character_credits,character_died_in,concept_credits,cover_date,description,has_staff_review,id,issue_number,location_credits,name,object_credits,person_credits,story_arc_credits,team_credits,team_disbanded_in,volume,combined_description
1408,"[Alfred_Pennyworth, Barbara_Gordon, Batman, De...",[],[],1997-04-01,"On his deathbed, a vengeful mob boss puts out ...",False,113178,708,[Gotham_City],The Death Lottery Part One : Heart of Glass,[],"[Android_Images, Bill_Sienkiewicz, Chuck_Dixon...",[],"[Batman_and_Robin, Gotham_City_Police_Department]",[],[Detective_Comics],"On his deathbed, a vengeful mob boss puts out ..."
2928,"[First_of_the_Fallen, John_Constantine, Kit_Ry...",[Nigel_Archer],"[Sex, Vertigo]",1994-10-24,As Constantine's final confrontation with Sata...,False,39834,82,[London],"Rake At the Gates of Hell, Part Five",[],"[Clem_Robins, Garth_Ennis, Glenn_Fabry, Julie_...",[],[],[],[Hellblazer],As Constantine's final confrontation with Sata...
274,"[Alfred_Pennyworth, Batman, Hush]",[],[],2005-09-01,Riley a normal random guy who is a photographe...,False,141753,67,[Gotham_City],The Life of Riley,[],"[A._J._Lieberman, Alvaro_Lopez, Guy_Major, Mat...",[],[],[],[Batman:_Gotham_Knights],Riley a normal random guy who is a photographe...


In [16]:
df['character_credits'][350]

['Bat-Mite', 'Batman', 'Dick_Grayson']

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6767 entries, 0 to 7060
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   character_credits     6767 non-null   object        
 1   character_died_in     6767 non-null   object        
 2   concept_credits       6767 non-null   object        
 3   cover_date            6767 non-null   datetime64[ns]
 4   description           6767 non-null   object        
 5   has_staff_review      6767 non-null   object        
 6   id                    6767 non-null   int64         
 7   issue_number          6767 non-null   object        
 8   location_credits      6767 non-null   object        
 9   name                  6767 non-null   object        
 10  object_credits        6767 non-null   object        
 11  person_credits        6767 non-null   object        
 12  story_arc_credits     6767 non-null   object        
 13  team_credits          6

In [18]:
df.to_csv("data/final/final_dataset_clean.csv", index=False)